# Part 5: Dataset Combination and Street Risk Level Visualization 

In [1]:
import pandas as pd
import numpy as np
import datetime

In [8]:
df = pd.read_csv('adtc.csv')
df['Date of Count'] = pd.to_datetime(df['Date of Count'], format = '%m/%d/%Y') #convert to appropriate datetime format

df['Day'] = df['Date of Count'].dt.weekday #extract 'Day' variable from date
df['Day'].unique()

array([3, 1, 2, 4, 0, 6], dtype=int64)

### 1. Extract Start and End Coordinates of Each Chicago Street Segment (According to Geospatial Data)

In [18]:
import re
streets = pd.read_csv('chicago_streets.csv')

long1 = []
lat1 = []
long2 = []
lat2 = []

for i in range(len(streets)): #extract starting and ending coordinates of each street segment
    if i % 10000 == 0:
        print(i)
    x = float(re.split(',| ', streets['48'][i].replace('(',''))[1])
    long1.append(x)
    x = float(re.split(',| ', streets['48'][i].replace('(',''))[2])
    lat1.append(x)
    x = float(re.split(',| ', streets['48'][i].replace(')',''))[-2])
    long2.append(x)
    x = float(re.split(',| ', streets['48'][i].replace(')',''))[-1])
    lat2.append(x)

streets['longitude1'] = pd.DataFrame(long1)
streets['latitude1'] = pd.DataFrame(lat1)
streets['longitude2'] = pd.DataFrame(long2)
streets['latitude2'] = pd.DataFrame(lat2)

streets['Traffic Volume Count Location Address'] = np.nan
streets['Street'] = np.nan
streets['Date of Count'] = np.nan
streets['Total Passing Vehicle Volume'] = np.nan
streets['Vehicle Volume By Each Direction of Traffic'] = np.nan
streets['Latitude'] = np.nan
streets['Longitude'] = np.nan

streets

0
10000
20000
30000
40000
50000


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,latitude1,longitude2,latitude2,Traffic Volume Count Location Address,Street,Date of Count,Total Passing Vehicle Volume,Vehicle Volume By Each Direction of Traffic,Latitude,Longitude
0,0,25190,11397,106439,N,ELSTON,AVE,NaN,495,4900,...,41.969599,-87.743500,41.970163,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,25159,10764,113102,N,ROCKWELL,ST,NaN,985,2222,...,41.922431,-87.692797,41.923125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,14645,14646,131228,W,69TH,ST,NaN,1955,5,...,41.769192,-87.625407,41.769189,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,19722,19723,139366,W,95TH,ST,NaN,2009,931,...,41.721417,-87.648074,41.721401,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,18436,10868,115772,S,DESPLAINES,ST,NaN,1317,1601,...,41.859970,-87.642913,41.859109,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,32295,9064,113922,N,PARIS,AVE,NaN,941,3500,...,41.942824,-87.830295,41.944929,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,105,36009,110104,N,SACRAMENTO,AVE,NaN,991,5700,...,41.984914,-87.704131,41.986732,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,19632,967,143722,NaN,NaN,NaN,NaN,1,0,...,42.000869,-87.919879,42.000251,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,22565,11682,128072,E,67TH,ST,NaN,159,1632,...,41.773345,-87.583861,41.773361,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,17154,36251,112223,N,MOBILE,AVE,NaN,848,1800,...,41.912818,-87.782180,41.914642,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2. Combining Average Daily Traffic Density Data Set and Chicago Street Geospatial Data Using Latitude and Longitude

In [19]:
# finding average daily traffic density for major road segments

for i in range(len(streets)):
    if i % 1000 == 0:
        print(i)
    mask = \
    ((streets.loc[i,'longitude1'] >= df.loc[:,'Longitude']) & (streets.loc[i,'longitude2'] <= df.loc[:,'Longitude']) & (streets.loc[i,'latitude1'] >= df.loc[:,'Latitude']) & (streets.loc[i,'latitude2'] <= df.loc[:,'Latitude'])) | \
    ((streets.loc[i,'longitude1'] >= df.loc[:,'Longitude']) & (streets.loc[i,'longitude2'] <= df.loc[:,'Longitude']) & (streets.loc[i,'latitude1'] <= df.loc[:,'Latitude']) & (streets.loc[i,'latitude2'] >= df.loc[:,'Latitude'])) | \
    ((streets.loc[i,'longitude1'] <= df.loc[:,'Longitude']) & (streets.loc[i,'longitude2'] >= df.loc[:,'Longitude']) & (streets.loc[i,'latitude1'] >= df.loc[:,'Latitude']) & (streets.loc[i,'latitude2'] <= df.loc[:,'Latitude'])) | \
    ((streets.loc[i,'longitude1'] <= df.loc[:,'Longitude']) & (streets.loc[i,'longitude2'] >= df.loc[:,'Longitude']) & (streets.loc[i,'latitude1'] <= df.loc[:,'Latitude']) & (streets.loc[i,'latitude2'] >= df.loc[:,'Latitude'])) 
    
    if len(df[mask]) > 0:
        streets.iloc[i,-7:] = np.array(df[mask][['Traffic Volume Count Location  Address', 'Street', 'Date of Count', 'Total Passing Vehicle Volume', 'Vehicle Volume By Each Direction of Traffic','Latitude','Longitude']])[0]       

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000


In [28]:
print('Number of major street segments with traffic density:', len(streets[streets.Latitude > 0]))
print('Percent of streets with traffic density:', np.around(len(streets[streets.Latitude > 0])/len(streets) * 100,2),'%')
streets.head()


Number of major street segments with traffic density: 1250
Percent of streets with traffic density: 2.24 %


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,latitude1,longitude2,latitude2,Traffic Volume Count Location Address,Street,Date of Count,Total Passing Vehicle Volume,Vehicle Volume By Each Direction of Traffic,Latitude,Longitude
0,0,25190,11397,106439,N,ELSTON,AVE,NaN,495,4900,...,41.969599,-87.743500,41.970163,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,25159,10764,113102,N,ROCKWELL,ST,NaN,985,2222,...,41.922431,-87.692797,41.923125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,14645,14646,131228,W,69TH,ST,NaN,1955,5,...,41.769192,-87.625407,41.769189,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,19722,19723,139366,W,95TH,ST,NaN,2009,931,...,41.721417,-87.648074,41.721401,948 West,95th Street,2006-03-22 00:00:00,29100.0,East Bound: 14900 / West Bound: 14200,41.721409,-87.647429
4,4,18436,10868,115772,S,DESPLAINES,ST,NaN,1317,1601,...,41.859970,-87.642913,41.859109,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Impute Average Daily Traffic Density Data using k Nearest Neighbors (k = 3)

In [87]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

knn = KNeighborsClassifier(n_neighbors=3)
scaler = StandardScaler()

not_null = streets[streets.loc[:,'Total Passing Vehicle Volume'] > 0].index
y = np.asarray(streets.loc[not_null, 'Total Passing Vehicle Volume'])
X = np.asarray(streets.iloc[not_null,51:55])
scaler.fit(X)
X = scaler.transform(X) 

null = streets[~(streets.loc[:,'Total Passing Vehicle Volume'] > 0)].index
X_predict = np.asarray(streets.iloc[null,51:55])
scaler.fit(X_predict)
X_predict = scaler.transform(X_predict) 

knn.fit(X,y)

y_predict = knn.predict(X_predict)
y_predict[:20]

array([20200., 12800.,  4200.,  8500., 16600., 26700., 19300., 12500.,
       13200.,  8400.,  5500.,  8200., 11100., 20200., 18000.,  9700.,
        3900., 22100., 11700.,  6700.])

In [94]:
streets.loc[null,'Total Passing Vehicle Volume'] = y_predict
streets.to_csv('streets_imputed_adtc.csv')

### 4. Combining Chicago Traffic Crash Dataset with Imputed ADTD Data (20/11/2018)

In [326]:
latest_crash = pd.read_csv('crash_241118.csv')
latest_crash

,Unnamed: 0,RD_NO,WEATHER_CONDITION,INJURIES_TOTAL,DAMAGE,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_HOUR,LIGHTING_CONDITION,MANEUVER,TRAFFICWAY_TYPE,PRIM_CONTRIBUTORY_CAUSE,POSTED_SPEED_LIMIT,LATITUDE,LONGITUDE,LOCATION,INJURIES_PER_PASSENGER,DANGER_SCORE,COMBINED_DANGER_SCORE
0,0,AJ101349,CLEAR,0.0,2,2,1,9,DAYLIGHT,CHANGING LANES,PARKING LOT,IMPROPER LANE USAGE,30,41.831296,-87.614926,POINT (-87.614925683354 41.831296076845),0.00,1.0,1.0
1,1,AJ101349,CLEAR,0.0,2,2,1,9,DAYLIGHT,STRAIGHT AHEAD,PARKING LOT,IMPROPER LANE USAGE,30,41.831296,-87.614926,POINT (-87.614925683354 41.831296076845),0.00,1.0,1.0
2,2,AJ103671,CLEAR,0.0,2,4,1,8,DAYLIGHT,PASSING/OVERTAKING,NOT DIVIDED,UNABLE TO DETERMINE,35,42.012292,-87.683228,POINT (-87.683227657543 42.012292006227),0.00,1.0,1.0
3,3,AJ103671,CLEAR,0.0,2,4,1,8,DAYLIGHT,OTHER,NOT DIVIDED,UNABLE TO DETERMINE,35,42.012292,-87.683228,POINT (-87.683227657543 42.012292006227),0.00,1.0,1.0
4,4,AJ114251,CLOUDY/OVERCAST,0.0,3,6,1,10,DAYLIGHT,PASSING/OVERTAKING,NOT DIVIDED,IMPROPER LANE USAGE,30,41.872058,-87.745079,POINT (-87.745079343101 41.872058050459),0.00,1.0,2.0
5,5,AJ114251,CLOUDY/OVERCAST,0.0,3,6,1,10,DAYLIGHT,CHANGING LANES,NOT DIVIDED,IMPROPER LANE USAGE,30,41.872058,-87.745079,POINT (-87.745079343101 41.872058050459),0.00,1.0,2.0
6,6,AJ123519,CLEAR,0.0,3,6,1,19,"DARKNESS, LIGHTED ROAD",STRAIGHT AHEAD,NOT DIVIDED,FAILING TO YIELD RIGHT-OF-WAY,15,41.736803,-87.587058,POINT (-87.587057964619 41.736802906927),0.00,1.0,2.0
7,7,AJ123519,CLEAR,0.0,3,6,1,19,"DARKNESS, LIGHTED ROAD",STRAIGHT AHEAD,NOT DIVIDED,FAILING TO YIELD RIGHT-OF-WAY,15,41.736803,-87.587058,POINT (-87.587057964619 41.736802906927),0.00,1.0,2.0
8,9,AJ390611,CLEAR,0.0,3,1,8,19,DARKNESS,UNKNOWN/NA,ONE-WAY,IMPROPER BACKING,30,41.741095,-87.561352,POINT (-87.561352197263 41.741094905698),0.00,1.0,2.0
9,10,HA102649,CLEAR,0.0,2,1,1,13,DAYLIGHT,BACKING,ONE-WAY,UNABLE TO DETERMINE,30,41.869674,-87.632368,POINT (-87.632367670736 41.86967388968),0.00,1.0,1.0


In [332]:
columns = list(orig_streets.columns)
columns[-12] = 'Geospatial'
orig_streets.columns = columns
orig_streets.iloc[:,-12:]

,Geospatial,longitude1,latitude1,longitude2,latitude2,Traffic Volume Count Location Address,Street,Date of Count,Total Passing Vehicle Volume,Vehicle Volume By Each Direction of Traffic,Latitude,Longitude
0,LINESTRING (-87.7425801790052 41.9695988260068...,-87.7426,41.9696,-87.7435,41.9702,NaN,NaN,NaN,26900,NaN,NaN,NaN
1,"LINESTRING (-87.692778179279 41.9224311746296,...",-87.6928,41.9224,-87.6928,41.9231,NaN,NaN,NaN,23300,NaN,NaN,NaN
2,LINESTRING (-87.6251142600668 41.7691920054002...,-87.6251,41.7692,-87.6254,41.7692,NaN,NaN,NaN,14400,NaN,NaN,NaN
3,LINESTRING (-87.6468546537901 41.7214167079522...,-87.6469,41.7214,-87.6481,41.7214,948 West,95th Street,2006-03-22 00:00:00,29100,East Bound: 14900 / West Bound: 14200,41.7214,-87.6474
4,LINESTRING (-87.6429377066171 41.8599702379946...,-87.6429,41.86,-87.6429,41.8591,NaN,NaN,NaN,15866.7,NaN,NaN,NaN
5,LINESTRING (-87.8302136472567 41.9428238421412...,-87.8302,41.9428,-87.8303,41.9449,NaN,NaN,NaN,27833.3,NaN,NaN,NaN
6,LINESTRING (-87.7040700277798 41.9849144909461...,-87.7041,41.9849,-87.7041,41.9867,NaN,NaN,NaN,18600,NaN,NaN,NaN
7,LINESTRING (-87.9198857619985 42.0008692340929...,-87.9199,42.0009,-87.9199,42.0003,NaN,NaN,NaN,25333.3,NaN,NaN,NaN
8,LINESTRING (-87.5850806141203 41.7733453641521...,-87.5851,41.7733,-87.5839,41.7734,NaN,NaN,NaN,36900,NaN,NaN,NaN
9,LINESTRING (-87.7825729212642 41.9128179698777...,-87.7826,41.9128,-87.7822,41.9146,NaN,NaN,NaN,16500,NaN,NaN,NaN


In [333]:
latest_crash['Geospatial'] = np.nan
latest_crash['longitude1'] = np.nan
latest_crash['latitude1'] = np.nan
latest_crash['longitude2'] = np.nan
latest_crash['latitude2'] = np.nan
latest_crash['Total Passing Vehicle Volume'] = np.nan

In [397]:
crash = latest_crash
streets = orig_streets

In [368]:
streets.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', '0', '1', '2', '3', '4', '5', '6', '7',
       '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',
       '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43',
       '44', '45', '46', '47', 'Geospatial', 'longitude1', 'latitude1',
       'longitude2', 'latitude2', 'Traffic Volume Count Location Address',
       'Street', 'Date of Count', 'Total Passing Vehicle Volume',
       'Vehicle Volume By Each Direction of Traffic', 'Latitude', 'Longitude'],
      dtype='object')

In [400]:
latest_crash.columns

Index(['Unnamed: 0', 'RD_NO', 'WEATHER_CONDITION', 'INJURIES_TOTAL', 'DAMAGE',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'CRASH_HOUR', 'LIGHTING_CONDITION',
       'MANEUVER', 'TRAFFICWAY_TYPE', 'PRIM_CONTRIBUTORY_CAUSE',
       'POSTED_SPEED_LIMIT', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'INJURIES_PER_PASSENGER', 'DANGER_SCORE', 'COMBINED_DANGER_SCORE',
       'Geospatial', 'longitude1', 'latitude1', 'longitude2', 'latitude2',
       'Total Passing Vehicle Volume'],
      dtype='object')

In [398]:
crash_index = crash.index
streets_index = streets.index

streets = np.asarray(streets)
crash = np.asarray(crash)

for i in range(len(crash)):
    if i % 5000 == 0:
        print(i)
    mask = \
    ((crash[i,-11] >= streets[:,-11]) & (crash[i,-11] <= streets[:,-9]) & (crash[i,-12] >= streets[:,-10]) & (crash[i,-12] <= streets[:,-8])) | \
    ((crash[i,-11] >= streets[:,-11]) & (crash[i,-11] <= streets[:,-9]) & (crash[i,-12] <= streets[:,-10]) & (crash[i,-12] >= streets[:,-8])) | \
    ((crash[i,-11] <= streets[:,-11]) & (crash[i,-11] >= streets[:,-9]) & (crash[i,-12] >= streets[:,-10]) & (crash[i,-12] <= streets[:,-8])) | \
    ((crash[i,-11] <= streets[:,-11]) & (crash[i,-11] >= streets[:,-9]) & (crash[i,-12] <= streets[:,-10]) & (crash[i,-12] >= streets[:,-8]))
    
    if len(streets[mask]) > 0:
        crash[i,-6:] = streets[mask][0][[-12,-11,-10,-9,-8,-4]]

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000
310000
315000
320000
325000
330000
335000
340000
345000
350000
355000
360000
365000


In [401]:
new_crash = pd.DataFrame(crash, columns = latest_crash.columns)
new_crash

,Unnamed: 0,RD_NO,WEATHER_CONDITION,INJURIES_TOTAL,DAMAGE,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_HOUR,LIGHTING_CONDITION,MANEUVER,...,LOCATION,INJURIES_PER_PASSENGER,DANGER_SCORE,COMBINED_DANGER_SCORE,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,0,AJ101349,CLEAR,0,2,2,1,9,DAYLIGHT,CHANGING LANES,...,POINT (-87.614925683354 41.831296076845),0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AJ101349,CLEAR,0,2,2,1,9,DAYLIGHT,STRAIGHT AHEAD,...,POINT (-87.614925683354 41.831296076845),0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AJ103671,CLEAR,0,2,4,1,8,DAYLIGHT,PASSING/OVERTAKING,...,POINT (-87.683227657543 42.012292006227),0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AJ103671,CLEAR,0,2,4,1,8,DAYLIGHT,OTHER,...,POINT (-87.683227657543 42.012292006227),0,1,1,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AJ114251,CLOUDY/OVERCAST,0,3,6,1,10,DAYLIGHT,PASSING/OVERTAKING,...,POINT (-87.745079343101 41.872058050459),0,1,2,LINESTRING (-87.7449325601314 41.8720723231659...,-87.7449,41.8721,-87.7459,41.8721,26800
5,5,AJ114251,CLOUDY/OVERCAST,0,3,6,1,10,DAYLIGHT,CHANGING LANES,...,POINT (-87.745079343101 41.872058050459),0,1,2,LINESTRING (-87.7449325601314 41.8720723231659...,-87.7449,41.8721,-87.7459,41.8721,26800
6,6,AJ123519,CLEAR,0,3,6,1,19,"DARKNESS, LIGHTED ROAD",STRAIGHT AHEAD,...,POINT (-87.587057964619 41.736802906927),0,1,2,NaN,NaN,NaN,NaN,NaN,NaN
7,7,AJ123519,CLEAR,0,3,6,1,19,"DARKNESS, LIGHTED ROAD",STRAIGHT AHEAD,...,POINT (-87.587057964619 41.736802906927),0,1,2,NaN,NaN,NaN,NaN,NaN,NaN
8,9,AJ390611,CLEAR,0,3,1,8,19,DARKNESS,UNKNOWN/NA,...,POINT (-87.561352197263 41.741094905698),0,1,2,NaN,NaN,NaN,NaN,NaN,NaN
9,10,HA102649,CLEAR,0,2,1,1,13,DAYLIGHT,BACKING,...,POINT (-87.632367670736 41.86967388968),0,1,1,NaN,NaN,NaN,NaN,NaN,NaN


In [409]:
print(new_crash['Total Passing Vehicle Volume'].notnull().sum()/len(new_crash) * 100, '% of data filled.')

16.022189991559937 % of data filled.


In [64]:
crash.to_csv('crash_geospatial.csv')

In [58]:
crash.loc[1:100,['LATITUDE','LONGITUDE','Geospatial','longitude1','latitude1','longitude2','latitude2']]

,LATITUDE,LONGITUDE,Geospatial,longitude1,latitude1,longitude2,latitude2
1,41.778544,-87.741895,NaN,NaN,NaN,NaN,NaN
2,41.735834,-87.667988,NaN,NaN,NaN,NaN,NaN
3,41.659214,-87.617189,NaN,NaN,NaN,NaN,NaN
4,41.939006,-87.727036,NaN,NaN,NaN,NaN,NaN
5,41.791396,-87.715882,NaN,NaN,NaN,NaN,NaN
6,41.706484,-87.642967,NaN,NaN,NaN,NaN,NaN
7,41.747439,-87.666860,NaN,NaN,NaN,NaN,NaN
8,41.800964,-87.689056,NaN,NaN,NaN,NaN,NaN
9,41.922306,-87.746287,NaN,NaN,NaN,NaN,NaN
10,41.755795,-87.674022,NaN,NaN,NaN,NaN,NaN


### 5. Revisiting and Revising Imputed ADTD Data (24/11/2018)

In [184]:
import pandas as pd
import numpy as np

crash = pd.read_csv('crash_geospatial.csv')

In [185]:
crash.iloc[:50,-14:]

,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,CRASH DAY,CRASH HOUR,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,3,1,9,41.890896,-87.623981,POINT (-87.62398101673 41.89089644229),6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,3,1,9,41.778544,-87.741895,POINT (-87.741895354166 41.778544187675),6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,9,41.735834,-87.667988,POINT (-87.667987863621 41.735833895809),6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1,9,41.659214,-87.617189,POINT (-87.617189092757 41.659213513068),6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2,1,9,41.939006,-87.727036,POINT (-87.727035635333 41.939006278197),6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,1,1,9,41.791396,-87.715882,POINT (-87.715882081132 41.791396122717),6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1,1,9,41.706484,-87.642967,POINT (-87.64296683145 41.706484273757),6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,1,1,9,41.747439,-87.666860,POINT (-87.666860014686 41.747438635053),6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,1,1,9,41.800964,-87.689056,POINT (-87.689055833949 41.800964200195),6.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,0,1,9,41.922306,-87.746287,POINT (-87.746287459891 41.922305982189),6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
orig_streets = pd.read_csv('chicago_streets_adtc')

In [255]:
df_test_X = np.asarray(orig_streets[orig_streets['Street'].isna()].iloc[:,[-11,-10,-9,-8]])
df_test_y = np.asarray(orig_streets[orig_streets['Street'].isna()].iloc[:,-4])
test_long = ((df_test_X[:,0] + df_test_X[:,2])/2).reshape(len(df_test_X),1)
test_lat = ((df_test_X[:,1] + df_test_X[:,3])/2).reshape(len(df_test_X),1)

In [278]:
df_train_X = np.asarray(orig_streets[~orig_streets['Street'].isna()].iloc[:,[-11,-10,-9,-8]])
df_train_y = pd.DataFrame(orig_streets[~orig_streets['Street'].isna()].iloc[:,-4])

train_long = ((df_train_X[:,0] + df_train_X[:,2])/2).reshape(len(df_train_X),1)
train_lat = ((df_train_X[:,1] + df_train_X[:,3])/2).reshape(len(df_train_X),1)

In [259]:
train = np.concatenate((train_long, train_lat), axis = 1)
test = np.concatenate((test_long, test_lat), axis = 1)

In [295]:
K = 3 
predicted_y = []

for j in range(len(test)):
    dist = list(np.sqrt(np.sum((test[j,:] - train)**2, axis = 1)))
    k_smallest_y = []

    df_train_ydrop = df_train_y.copy()
    for i in range(K):
        predict = df_train_ydrop.iloc[np.argmin(dist),:][0]
        k_smallest_y.append([predict])
        df_train_ydrop.drop(df_train_ydrop.index[np.argmin(dist)], inplace = True)
        dist.pop(np.argmin(dist))

    k_mean_y = np.mean(k_smallest_y)
    predicted_y.append([k_mean_y])

In [314]:
pd.options.mode.chained_assignment = None
street_missing_index = list(orig_streets[orig_streets['Street'].isna()].index)

original_streets = np.asarray(orig_streets)
j = 0

for i in street_missing_index:
    if j % 10000 == 0:
        print(j)
    original_streets[i,-4] = predicted_y[j][0]
    j += 1

orig_streets = pd.DataFrame(original_streets, columns = orig_streets.columns)

0
10000
20000
30000
40000
50000


In [317]:
orig_streets.to_csv('new_imputed_streets.csv')

In [1]:
import numpy as np
import pandas as pd

In [6]:
orig_streets = pd.read_csv('new_imputed_streets.csv')
street_columns = list(orig_streets.columns)
street_columns[-12] = 'Geospatial'
orig_streets.columns = street_columns
print(orig_streets.columns)
orig_streets.head()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '0', '1', '2', '3', '4',
       '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41',
       '42', '43', '44', '45', '46', '47', 'Geospatial', 'longitude1',
       'latitude1', 'longitude2', 'latitude2',
       'Traffic Volume Count Location Address', 'Street', 'Date of Count',
       'Total Passing Vehicle Volume',
       'Vehicle Volume By Each Direction of Traffic', 'Latitude', 'Longitude'],
      dtype='object')


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,0,1,2,3,4,5,6,...,latitude1,longitude2,latitude2,Traffic Volume Count Location Address,Street,Date of Count,Total Passing Vehicle Volume,Vehicle Volume By Each Direction of Traffic,Latitude,Longitude
0,0,0,0,25190,11397,106439,N,ELSTON,AVE,NaN,...,41.969599,-87.743500,41.970163,NaN,NaN,NaN,26900.000000,NaN,NaN,NaN
1,1,1,1,25159,10764,113102,N,ROCKWELL,ST,NaN,...,41.922431,-87.692797,41.923125,NaN,NaN,NaN,23300.000000,NaN,NaN,NaN
2,2,2,2,14645,14646,131228,W,69TH,ST,NaN,...,41.769192,-87.625407,41.769189,NaN,NaN,NaN,14400.000000,NaN,NaN,NaN
3,3,3,3,19722,19723,139366,W,95TH,ST,NaN,...,41.721417,-87.648074,41.721401,948 West,95th Street,2006-03-22 00:00:00,29100.000000,East Bound: 14900 / West Bound: 14200,41.721409,-87.647429
4,4,4,4,18436,10868,115772,S,DESPLAINES,ST,NaN,...,41.859970,-87.642913,41.859109,NaN,NaN,NaN,15866.666667,NaN,NaN,NaN


### 6. Imputing Chicago Traffic Crash Dataset (missing ADTD values) by Finding Closest Street with Non-Missing ADTD Value

In [3]:
new_crash = pd.read_csv('new_crash_251118.csv')
print(new_crash.columns)
new_crash.head(6)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'RD_NO', 'WEATHER_CONDITION',
       'INJURIES_TOTAL', 'DAMAGE', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'CRASH_HOUR', 'LIGHTING_CONDITION', 'MANEUVER', 'TRAFFICWAY_TYPE',
       'PRIM_CONTRIBUTORY_CAUSE', 'POSTED_SPEED_LIMIT', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'INJURIES_PER_PASSENGER', 'DANGER_SCORE',
       'COMBINED_DANGER_SCORE', 'Geospatial', 'longitude1', 'latitude1',
       'longitude2', 'latitude2', 'Total Passing Vehicle Volume'],
      dtype='object')


,Unnamed: 0,Unnamed: 0.1,RD_NO,WEATHER_CONDITION,INJURIES_TOTAL,DAMAGE,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_HOUR,LIGHTING_CONDITION,...,LOCATION,INJURIES_PER_PASSENGER,DANGER_SCORE,COMBINED_DANGER_SCORE,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,0,0,AJ101349,CLEAR,0.0,2,2,1,9,DAYLIGHT,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,AJ101349,CLEAR,0.0,2,2,1,9,DAYLIGHT,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,AJ103671,CLEAR,0.0,2,4,1,8,DAYLIGHT,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,AJ103671,CLEAR,0.0,2,4,1,8,DAYLIGHT,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,AJ114251,CLOUDY/OVERCAST,0.0,3,6,1,10,DAYLIGHT,...,POINT (-87.745079343101 41.872058050459),0.0,1.0,2.0,LINESTRING (-87.7449325601314 41.8720723231659...,-87.744933,41.872072,-87.745877,41.872058,26800.0


In [4]:
unlabeled = new_crash[new_crash['Geospatial'].isna()][['LATITUDE','LONGITUDE']]
print(unlabeled.shape)
unlabeled.head(10)

(307452, 2)


,LATITUDE,LONGITUDE
0,41.831296,-87.614926
1,41.831296,-87.614926
2,42.012292,-87.683228
3,42.012292,-87.683228
6,41.736803,-87.587058
7,41.736803,-87.587058
8,41.741095,-87.561352
9,41.869674,-87.632368
10,41.869674,-87.632368
11,41.809408,-87.607552


In [8]:
streets = orig_streets[['Geospatial','longitude1','latitude1','longitude2','latitude2','Total Passing Vehicle Volume']]
streets.head()

,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,LINESTRING (-87.7425801790052 41.9695988260068...,-87.742580,41.969599,-87.743500,41.970163,26900.000000
1,"LINESTRING (-87.692778179279 41.9224311746296,...",-87.692778,41.922431,-87.692797,41.923125,23300.000000
2,LINESTRING (-87.6251142600668 41.7691920054002...,-87.625114,41.769192,-87.625407,41.769189,14400.000000
3,LINESTRING (-87.6468546537901 41.7214167079522...,-87.646855,41.721417,-87.648074,41.721401,29100.000000
4,LINESTRING (-87.6429377066171 41.8599702379946...,-87.642938,41.859970,-87.642913,41.859109,15866.666667


In [10]:
def closest_street(streets, unlabeled):
    x1 = np.asarray(streets.iloc[:,1])
    y1 = np.asarray(streets.iloc[:,2])
    x2 = np.asarray(streets.iloc[:,3])
    y2 = np.asarray(streets.iloc[:,4])
    streets = np.asarray(streets)

    x3 = np.asarray(unlabeled['LONGITUDE'])
    y3 = np.asarray(unlabeled['LATITUDE'])

    center_x = (x2 + x1)/2 #find center of streets
    center_y = (y2 + y1)/2
    closest = []

    for i in range(len(x3)):
        if i % 5000 == 0:
            print(i)
        distance = ((x3[i] - center_x) **2 + (y3[i] - center_y) ** 2)**0.5
        min_index = distance.argmin()
        data = streets[min_index, [0,1,2,3,4,5]] #pay attention to the columns you are extracting!
        closest.append(data)
    closest = np.asarray(closest).reshape(len(x3),6)
    return closest 

In [11]:
closest = closest_street(streets, unlabeled)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
235000
240000
245000
250000
255000
260000
265000
270000
275000
280000
285000
290000
295000
300000
305000


In [14]:
unlabeled.index

Int64Index([     0,      1,      2,      3,      6,      7,      8,      9,
                10,     11,
            ...
            366098, 366101, 366102, 366103, 366104, 366105, 366106, 366107,
            366109, 366110],
           dtype='int64', length=307452)

In [16]:
pd.options.mode.chained_assignment = None
unlabeled_index = list(unlabeled.index)

z = np.asarray(new_crash)
j = 0

for i in unlabeled_index:
    if j % 10000 == 0:
        print(j)
    z[i,-6:] = closest[j,:]
    j += 1

new_crash = pd.DataFrame(z, columns = new_crash.columns)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [20]:
print('Number of unique streets:', len(new_crash['Geospatial'].unique()))
new_crash.isna().sum()

Number of unique streets: 35171


Unnamed: 0                      0
Unnamed: 0.1                    0
RD_NO                           0
WEATHER_CONDITION               0
INJURIES_TOTAL                  0
DAMAGE                          0
CRASH_DAY_OF_WEEK               0
CRASH_MONTH                     0
CRASH_HOUR                      0
LIGHTING_CONDITION              0
MANEUVER                        0
TRAFFICWAY_TYPE                 0
PRIM_CONTRIBUTORY_CAUSE         0
POSTED_SPEED_LIMIT              0
LATITUDE                        0
LONGITUDE                       0
LOCATION                        0
INJURIES_PER_PASSENGER          0
DANGER_SCORE                    0
COMBINED_DANGER_SCORE           0
Geospatial                      0
longitude1                      0
latitude1                       0
longitude2                      0
latitude2                       0
Total Passing Vehicle Volume    0
dtype: int64

In [27]:
new_crash.to_csv('imputed_crash_251118.csv')

### 7. Investigating Fully-imputed Chicago Traffic Crash Dataset (26/11/2018)

In [2]:
import numpy as np
import pandas as pd

crash = pd.read_csv('imputed_crash_251118.csv')

In [3]:
print(crash.columns)
crash.head()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'RD_NO',
       'WEATHER_CONDITION', 'INJURIES_TOTAL', 'DAMAGE', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'CRASH_HOUR', 'LIGHTING_CONDITION', 'MANEUVER',
       'TRAFFICWAY_TYPE', 'PRIM_CONTRIBUTORY_CAUSE', 'POSTED_SPEED_LIMIT',
       'LATITUDE', 'LONGITUDE', 'LOCATION', 'INJURIES_PER_PASSENGER',
       'DANGER_SCORE', 'COMBINED_DANGER_SCORE', 'Geospatial', 'longitude1',
       'latitude1', 'longitude2', 'latitude2', 'Total Passing Vehicle Volume'],
      dtype='object')


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,RD_NO,WEATHER_CONDITION,INJURIES_TOTAL,DAMAGE,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_HOUR,...,LOCATION,INJURIES_PER_PASSENGER,DANGER_SCORE,COMBINED_DANGER_SCORE,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,0,0,0,AJ101349,CLEAR,0.0,2,2,1,9,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,LINESTRING (-87.6169664634471 41.8311001555208...,-87.616966,41.831100,-87.614485,41.831196,11800.000000
1,1,1,1,AJ101349,CLEAR,0.0,2,2,1,9,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,LINESTRING (-87.6169664634471 41.8311001555208...,-87.616966,41.831100,-87.614485,41.831196,11800.000000
2,2,2,2,AJ103671,CLEAR,0.0,2,4,1,8,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,LINESTRING (-87.6830529923006 42.0122200073731...,-87.683053,42.012220,-87.683086,42.012323,11533.333333
3,3,3,3,AJ103671,CLEAR,0.0,2,4,1,8,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,LINESTRING (-87.6830529923006 42.0122200073731...,-87.683053,42.012220,-87.683086,42.012323,11533.333333
4,4,4,4,AJ114251,CLOUDY/OVERCAST,0.0,3,6,1,10,...,POINT (-87.745079343101 41.872058050459),0.0,1.0,2.0,LINESTRING (-87.7449325601314 41.8720723231659...,-87.744933,41.872072,-87.745877,41.872058,26800.000000


In [4]:
days = {1:6, 2:0, 3:1, 4:2, 5:3, 6:4, 7:5}
crash['CRASH_DAY_OF_WEEK'] = crash['CRASH_DAY_OF_WEEK'].map(days)
crash.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,RD_NO,WEATHER_CONDITION,INJURIES_TOTAL,DAMAGE,CRASH_DAY_OF_WEEK,CRASH_MONTH,CRASH_HOUR,...,LOCATION,INJURIES_PER_PASSENGER,DANGER_SCORE,COMBINED_DANGER_SCORE,Geospatial,longitude1,latitude1,longitude2,latitude2,Total Passing Vehicle Volume
0,0,0,0,AJ101349,CLEAR,0.0,2,0,1,9,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,LINESTRING (-87.6169664634471 41.8311001555208...,-87.616966,41.831100,-87.614485,41.831196,11800.000000
1,1,1,1,AJ101349,CLEAR,0.0,2,0,1,9,...,POINT (-87.614925683354 41.831296076845),0.0,1.0,1.0,LINESTRING (-87.6169664634471 41.8311001555208...,-87.616966,41.831100,-87.614485,41.831196,11800.000000
2,2,2,2,AJ103671,CLEAR,0.0,2,2,1,8,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,LINESTRING (-87.6830529923006 42.0122200073731...,-87.683053,42.012220,-87.683086,42.012323,11533.333333
3,3,3,3,AJ103671,CLEAR,0.0,2,2,1,8,...,POINT (-87.683227657543 42.012292006227),0.0,1.0,1.0,LINESTRING (-87.6830529923006 42.0122200073731...,-87.683053,42.012220,-87.683086,42.012323,11533.333333
4,4,4,4,AJ114251,CLOUDY/OVERCAST,0.0,3,4,1,10,...,POINT (-87.745079343101 41.872058050459),0.0,1.0,2.0,LINESTRING (-87.7449325601314 41.8720723231659...,-87.744933,41.872072,-87.745877,41.872058,26800.000000


In [5]:
z = crash[['CRASH_DAY_OF_WEEK','CRASH_HOUR','Geospatial','Total Passing Vehicle Volume','COMBINED_DANGER_SCORE']]
z.head()

,CRASH_DAY_OF_WEEK,CRASH_HOUR,Geospatial,Total Passing Vehicle Volume,COMBINED_DANGER_SCORE
0,0,9,LINESTRING (-87.6169664634471 41.8311001555208...,11800.000000,1.0
1,0,9,LINESTRING (-87.6169664634471 41.8311001555208...,11800.000000,1.0
2,2,8,LINESTRING (-87.6830529923006 42.0122200073731...,11533.333333,1.0
3,2,8,LINESTRING (-87.6830529923006 42.0122200073731...,11533.333333,1.0
4,4,10,LINESTRING (-87.7449325601314 41.8720723231659...,26800.000000,2.0


In [8]:
x = z.groupby(['CRASH_DAY_OF_WEEK','CRASH_HOUR','Geospatial']).mean()
x.head()

Total Passing Vehicle Volume  \
CRASH_DAY_OF_WEEK CRASH_HOUR Geospatial                                                                         
0                 0          LINESTRING (-87.5411058811197 41.6536662455898,...                  11900.000000   
                             LINESTRING (-87.5413396523992 41.7032336358462,...                  11900.000000   
                             LINESTRING (-87.5483692193163 41.7519726845212,...                  10766.666667   
                             LINESTRING (-87.5526792777482 41.7317697832974,...                  26766.666667   
                             LINESTRING (-87.5554940046535 41.7336797256645,...                  19466.666667   

                                                                                 COMBINED_DANGER_SCORE  
CRASH_DAY_OF_WEEK CRASH_HOUR Geospatial                                                                 
0                 0          LINESTRING (-87.5411058811197 41.6536662455898,...                    2.0  
                             LINESTRING (-87.5413396523992 41.7032336358462,...                    2.0  
                             LINESTRING (-87.5483692193163 41.7519726845212,...                    2.0  
                             LINESTRING (-87.5526792777482 41.7317697832974,...                    2.0  
                             LINESTRING (-87.5554940046535 41.7336797256645,...                    2.0

In [9]:
x.reset_index(level = ['CRASH_DAY_OF_WEEK','CRASH_HOUR'], inplace = True)
x.head()

,CRASH_DAY_OF_WEEK,CRASH_HOUR,Total Passing Vehicle Volume,COMBINED_DANGER_SCORE
Geospatial,,,,
"LINESTRING (-87.5411058811197 41.6536662455898,-87.5407755503155 41.6536636471998,-87.5404820160663 41.6536625994619,-87.5402544473951 41.6536618031811,-87.539884554008 41.6536609230813)",0,0,11900.000000,2.0
"LINESTRING (-87.5413396523992 41.7032336358462,-87.5413419155131 41.7027573341982)",0,0,11900.000000,2.0
"LINESTRING (-87.5483692193163 41.7519726845212,-87.5481495296341 41.7519758277349)",0,0,10766.666667,2.0
"LINESTRING (-87.5526792777482 41.7317697832974,-87.5526651123318 41.731191809164,-87.5526532938483 41.7306878472356,-87.5526504440013 41.7305668544632,-87.5526473426971 41.7304354681309,-87.5526386255178 41.7300574862073)",0,0,26766.666667,2.0
"LINESTRING (-87.5554940046535 41.7336797256645,-87.5554823538081 41.7331955031181,-87.5554733851717 41.7329295872398,-87.5554674340205 41.7327059999726,-87.5554587168412 41.7323561812436,-87.555455783175 41.7322302012389,-87.5554470659958 41.7318495371062)",0,0,19466.666667,2.0


In [21]:
x['CRASH_DAY_OF_WEEK'] = pd.to_numeric(x['CRASH_DAY_OF_WEEK'], downcast = 'float')
x['CRASH_HOUR'] = pd.to_numeric(x['CRASH_HOUR'], downcast = 'float')
type(x['CRASH_HOUR'][0])

numpy.float32

In [26]:
y = pd.read_csv('Traffic_Crashes_-_Crashes.csv')
y.head()

,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,JB447178,NaN,09/23/2018 03:00:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,...,0.0,0.0,2.0,0.0,3,1,9,41.890896,-87.623981,POINT (-87.62398101673 41.89089644229)
1,JB447158,NaN,09/23/2018 03:00:00 AM,0,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,PARKING LOT,...,0.0,0.0,1.0,0.0,3,1,9,41.778544,-87.741895,POINT (-87.741895354166 41.778544187675)
2,JB447148,NaN,09/23/2018 02:25:00 AM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,...,0.0,0.0,3.0,0.0,2,1,9,41.735834,-87.667988,POINT (-87.667987863621 41.735833895809)
3,JB447133,NaN,09/23/2018 02:15:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,2.0,0.0,2,1,9,41.659214,-87.617189,POINT (-87.617189092757 41.659213513068)
4,JB447129,NaN,09/23/2018 02:02:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,...,0.0,0.0,2.0,0.0,2,1,9,41.939006,-87.727036,POINT (-87.727035635333 41.939006278197)


In [28]:
p = y[y['CRASH_HOUR'] == 2]
p.to_csv('monday_accidents.csv')